# There and Back Again: A Collaborative Learning Demonstration on MNIST and rotated-MNIST

## The Basic Ingredients

- Loading the dataset
- Visualizing the images
- Defining the model
- Running the training loop
- Evaluating the model

## Loading the dataset

In [114]:
import torch
import torchvision
import torchvision.transforms as tf

transform = tf.Compose([
    tf.ToTensor(),
    tf.Normalize(0.5, 0.5),
])

rotated_transform = tf.Compose([
    tf.ToTensor(),
    tf.Normalize(0.5, 0.5),
    tf.RandomRotation(45)
])

batch_size = 16

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
train_iter = iter(trainloader)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=True)
test_iter = iter(testloader)


rotated_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=rotated_transform)
rotated_trainloader = torch.utils.data.DataLoader(rotated_trainset, batch_size = batch_size, shuffle=True)
rotated_train_iter = iter(rotated_trainloader)

rotated_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=rotated_transform)
rotated_testloader = torch.utils.data.DataLoader(rotated_testset, batch_size = batch_size, shuffle=True)
rotated_test_iter = iter(rotated_testloader)

## Visualizing the dataset

In [115]:
import plotly.express as px

def imshow(im):
    if torch.min(im) < 0:
        im = im - torch.min(im)
        im = im / torch.max(im)


    ndims = len(im.shape)
    im = torch.squeeze(im)
    if len(im.shape) == ndims:
        im = im.permute(1,2,0)
    
    fig = px.imshow(im)

    fig.update_layout(margin={'t': 0, 'r': 0, 'b': 0, 'l': 0})

    if im.shape[0] == im.shape[1]:
        fig.update_layout(width=400, height=400)
    fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
    fig.show()

batch, labels = next(rotated_train_iter)
N = 16
imshow(torchvision.utils.make_grid(batch[:N, :, :, :], nrow = 4, normalize=True))
print([l.item() for l in labels])

[2, 2, 7, 8, 3, 2, 1, 0, 8, 8, 1, 0, 7, 1, 9, 5]


## Defining the Model

In [116]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): # Input size = 32
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, 5)
        self.conv2 = nn.Conv2d(3, 6, 5)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6 * 4 * 4, 80)
        self.fc2 = nn.Linear(80, 40)
        self.fc3 = nn.Linear(40, 10)

    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x 
        

net = Net()

## Initial Model Training

In [117]:
def train(net, optimizer, dataloader, num_epochs=2):
    loss_fn = torch.nn.CrossEntropyLoss()

    for e in range(num_epochs):
        cumulative_loss = 0
        loss_window = len(dataloader) // 10
        for i, (batch, labels) in enumerate(dataloader, 1):
            optimizer.zero_grad()

            preds = net(batch)
            loss = loss_fn(preds, labels)
            loss.backward()
            optimizer.step()

            cumulative_loss += loss

            if i % loss_window == 0:
                print(f"Loss at epoch {e+1}, batch {i}: {cumulative_loss / loss_window :.2f}")
                cumulative_loss = 0

In [118]:
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
num_epochs = 2

train(net, optimizer, trainloader, num_epochs)

Loss at epoch 1, batch 375: 0.92
Loss at epoch 1, batch 750: 0.28
Loss at epoch 1, batch 1125: 0.23
Loss at epoch 1, batch 1500: 0.18
Loss at epoch 1, batch 1875: 0.15
Loss at epoch 1, batch 2250: 0.13
Loss at epoch 1, batch 2625: 0.12
Loss at epoch 1, batch 3000: 0.12
Loss at epoch 1, batch 3375: 0.11
Loss at epoch 1, batch 3750: 0.11
Loss at epoch 2, batch 375: 0.11
Loss at epoch 2, batch 750: 0.10
Loss at epoch 2, batch 1125: 0.10
Loss at epoch 2, batch 1500: 0.09
Loss at epoch 2, batch 1875: 0.08
Loss at epoch 2, batch 2250: 0.09
Loss at epoch 2, batch 2625: 0.08
Loss at epoch 2, batch 3000: 0.08
Loss at epoch 2, batch 3375: 0.08
Loss at epoch 2, batch 3750: 0.09


In [119]:
torch.onnx.export(net, batch, "mnist_test_pretrained.onnx", do_constant_folding=False)

## Doing some inference

In [120]:
import numpy as np

images, labels = next(test_iter)

imshow(torchvision.utils.make_grid(images))

print(list(torch.argmax(net(images), axis=1).numpy()))
print([l.item() for l in labels])

[8, 6, 7, 9, 8, 7, 3, 2, 7, 0, 1, 9, 8, 3, 1, 3]
[8, 6, 7, 9, 8, 7, 3, 2, 1, 0, 1, 9, 8, 3, 1, 3]


## Evaluating the Model

In [121]:
def evaluate_model(net, dataloader, class_specific_performance=False):
    if not class_specific_performance:
        correct, total = (0, 0)
    else:        
        correct = {digit: 0 for digit in range(10)}
        total = {digit: 0 for digit in range(10)}

    with torch.no_grad():
        for (batch, labels) in dataloader:
            outputs = net(batch)
            preds = torch.argmax(outputs, 1)

            if not class_specific_performance:
                total += labels.shape[0]
                correct += torch.sum(preds == labels).item()
            else:
                for label, pred in zip(labels, preds):                
                    if pred == label:
                        correct[label.item()] += 1
                    total[label.item()] += 1
    return correct, total

In [122]:
correct, total = evaluate_model(net, testloader)

print(f"The model classified {correct} digits correctly, out of {total} total, for an accuracy of {correct * 100 / total :.2f}%")

The model classified 9780 digits correctly, out of 10000 total, for an accuracy of 97.80%


## But how does it perform in the rotated context?

In [123]:
correct, total = evaluate_model(net, rotated_testloader, class_specific_performance=True)
c = sum(correct.values())
t = sum(total.values())
print(f"The model classified {c} rotated digits correctly, out of {t} total, for an accuracy of {c * 100 / t :.2f}%")

The model classified 8090 rotated digits correctly, out of 10000 total, for an accuracy of 80.90%


Open Question: Why does this change on each run?

In [124]:
digits, counts = list(zip(*correct.items()))
fig = px.bar(x = digits, y=np.array(counts)/1000, labels={'x': "digit", 'y': "Classification Success Rate"}, range_y=(0, 1))
fig.update_layout(yaxis={"tickformat": ",.0%"}, xaxis={"tickmode": "linear"})
fig.show()

## Now let's train the model again, this time using rotated data, and store the updates efficiently

In [125]:
import onnx
from onnx2pytorch import ConvertModel

onnx_model = onnx.load("mnist_test_pretrained.onnx")
imported_pytorch_model = ConvertModel(onnx_model, experimental=True)

/Users/montyevans/opt/miniconda3/envs/raffellab/lib/python3.9/site-packages/onnx2pytorch/convert/model.py:147: UserWarning:

Using experimental implementation that allows 'batch_size > 1'.Batchnorm layers could potentially produce false outputs.



Annoyingly, there is no native Pytorch implementation to import an ONNX model. However, I found an external library that attempts to do this, though it produces some strange results. Specifically, it adds several parameters to the model's state dict that I don't understand: they concern the weight and bias of the ReLUs, which I don't understand, since ReLUs have neither of those things.

In [126]:
print(imported_pytorch_model.state_dict().keys())
print("\n ----- \n")
print(net.state_dict().keys())

odict_keys(['_initializer_conv1_weight', '_initializer_conv1_bias', '_initializer_conv2_weight', '_initializer_conv2_bias', '_initializer_fc1_weight', '_initializer_fc1_bias', '_initializer_fc2_weight', '_initializer_fc2_bias', '_initializer_fc3_weight', '_initializer_fc3_bias', 'Conv_onnx::Relu_11.weight', 'Conv_onnx::Relu_11.bias', 'Conv_onnx::Relu_14.weight', 'Conv_onnx::Relu_14.bias', 'Gemm_onnx::Relu_18.weight', 'Gemm_onnx::Relu_18.bias', 'Gemm_onnx::Relu_20.weight', 'Gemm_onnx::Relu_20.bias', 'Gemm_22.weight', 'Gemm_22.bias'])

 ----- 

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


Nevertheless, we persist. Let's evaluate the imported model's performance and verify that it has similar performance to the original.

In [127]:
correct_normal, total_normal = evaluate_model(imported_pytorch_model, testloader)
correct_rotated, total_rotated = evaluate_model(imported_pytorch_model, rotated_testloader)

print(f"The imported model classified {correct_normal} digits correctly, out of {total_normal} total, for an accuracy of {correct_normal * 100 / total_normal :.2f}%")
print(f"The imported model classified {correct_rotated} rotated digits correctly, out of {total_rotated} total, for an accuracy of {correct_rotated * 100 / total_rotated :.2f}%")

/Users/montyevans/opt/miniconda3/envs/raffellab/lib/python3.9/site-packages/torch/nn/functional.py:749: UserWarning:

Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.



KeyboardInterrupt: 

It does - great!

Now, there are two approaches we could take to non-dense fine tuning. To clarify terminology, we distinguish between a _parameter_ and a _parameter group_. A _parameter_ is a single scalar value that is trained and used by the model during inference to produce a result. When people refer to the size of large models (e.g. T5 has 11B parameters), they are referring to these scalar values.

A _parameter group_ is collection of parameters that has some conceptual and practical meaning as a unit of computation. Typical examples are the bias vector and the weight matrix of a linear layer, or the kernel of a 2D convolution. 

A typical training run (like the one we did at the start of this notebook) trains all possible parameters at every training step: every parameter within every parameter group. If you wanted to do less exhaustive training (and efficiently store the corresponding update), there are two natural approaches.

- Select some subset of the _parameter groups_, and only train them. We refer to this as _globally sparse_ training.
- Within each parameter group, select some subset of _parameters_, and only train them. We refer to this as _locally sparse_ training.

(You could, of course, combine both approaches).

To keep this test run simple, we take the second approach and train only the first convolutional layer, and the final dense layer.

## Sparsely fine-tuning the model

In [ ]:
from collections import defaultdict

def train_with_mask(net, optimizer, dataloader, num_epochs=2, parameter_masks = None):
    if parameter_masks is None: parameter_masks = defaultdict(lambda: 1) # If no masks are provided, train all parameters as normal

    loss_fn = torch.nn.CrossEntropyLoss()

    for e in range(num_epochs):
        flag = 0
        cumulative_loss = 0
        loss_window = len(dataloader) // 10
        for i, (batch, labels) in enumerate(dataloader, 1):
            optimizer.zero_grad()

            preds = net(batch)
            loss = loss_fn(preds, labels)
            loss.backward()

            # Mask out the parameters here
            for name, params in net.named_parameters():
                params.grad.data.copy_(params.grad.data * parameter_masks[name])
                # params.grad.data.copy_(params.grad.data * 0)

            optimizer.step()

            cumulative_loss += loss

            if i % loss_window == 0:
                print(f"Loss at epoch {e+1}, batch {i}: {cumulative_loss / loss_window :.2f}")
                cumulative_loss = 0

In [133]:
# We consider 'true' parameter group names to be the ones that the original Pytorch model uses in net.named_parameters(), which
# are also the names in the ONNX model. 

# When we import back to Pytorch, in the model's state_dict() these are a) duplicated and b) both names are changed: N -> (N', N''). The
# model only seems to consider the N' in named_parameters() though, so these are what get trained. However, N'' is easier to convert back to N,
# so we use it as an intermediary.


# Q: How to reliably go from e.g. Conv_onnx::Relu_11.bias -> conv1.weight, and conv1.weight -> Conv_onnx::Relu_11.bias
# A: Can we assume the parameters will always be in the same order?


def mangle_parameter_name(name, model, direction="forward"):

    param_names = list(model.state_dict().keys())
    L = int(len(param_names)/2)
    simple, complex = param_names[:L], param_names[L:]

    
    pairs = list(zip(simple, complex))
    lookup = {pair[0]: pair[1] for pair in pairs}
    rev_lookup = {pair[1]: pair[0] for pair in pairs}

    if direction == "forward":
        layer, component = name.split('.')
        middle = f"_initializer_{layer}_{component}"
        return lookup[middle]

    elif direction == "backward":
        middle = rev_lookup[name]
        junk = "_initializer_"
        name = middle[len(junk):]
        layer, component = name.split('_')
        return f"{layer}.{component}"
    else:
        raise NotImplementedError(f"Direction must be once of [forward, backward]. You provided {direction}.")

def get_parameter_masks(parameters_to_train):
    masks = defaultdict(lambda: 0)    
    
    to_train = [mangle_parameter_name(name, imported_pytorch_model) for name in parameters_to_train]
    for param_name in to_train: masks[param_name] = 1
    return masks

In [128]:
from copy import deepcopy

params_to_train = ["conv1.weight", "conv1.bias", "fc3.weight", "fc3.bias"]
masks = get_parameter_masks(params_to_train)

model2 = deepcopy(imported_pytorch_model)
optimizer = torch.optim.Adam(model2.parameters())
train_with_mask(model2, optimizer, rotated_trainloader, 1, parameter_masks=masks)

Loss at epoch 1, batch 375: 0.53
Loss at epoch 1, batch 750: 0.45
Loss at epoch 1, batch 1125: 0.41
Loss at epoch 1, batch 1500: 0.41
Loss at epoch 1, batch 1875: 0.40
Loss at epoch 1, batch 2250: 0.39
Loss at epoch 1, batch 2625: 0.40
Loss at epoch 1, batch 3000: 0.39
Loss at epoch 1, batch 3375: 0.38
Loss at epoch 1, batch 3750: 0.38


In [129]:
correct_normal, total_normal = evaluate_model(model2, testloader)
correct_rotated, total_rotated = evaluate_model(model2, rotated_testloader)

print(f"The imported model classified {correct_normal} digits correctly, out of {total_normal} total, for an accuracy of {correct_normal * 100 / total_normal :.2f}%")
print(f"The imported model classified {correct_rotated} rotated digits correctly, out of {total_rotated} total, for an accuracy of {correct_rotated * 100 / total_rotated :.2f}%")

The imported model classified 9690 digits correctly, out of 10000 total, for an accuracy of 96.90%
The imported model classified 8839 rotated digits correctly, out of 10000 total, for an accuracy of 88.39%


Success! The model improved its accuracy on the rotated digit set by 10% (and dropped 1% on the standard set, but omelettes and eggs...)

Let's check to see that the parameters we wanted to train actually changed, while the others stayed the same.

In [132]:
param_pairs = list(zip(net.named_parameters(), model2.named_parameters()))
for param in param_pairs:
    if torch.allclose(param[0][1], param[1][1]):
        print(f"{param[0][0]} i.e. {param[1][0]} did not change", end=' --- ')
        print("Goddamn") if param[1][0] in params_to_train else print("Yay!")
    else:
        print(f"{param[0][0]} i.e. {param[1][0]} changed", end=' --- ')
        print("Goddamn") if param[1][0] not in params_to_train else print("Yay!")

conv1.weight i.e. Conv_onnx::Relu_11.weight changed --- Yay!
conv1.bias i.e. Conv_onnx::Relu_11.bias changed --- Yay!
conv2.weight i.e. Conv_onnx::Relu_14.weight did not change --- Yay!
conv2.bias i.e. Conv_onnx::Relu_14.bias did not change --- Yay!
fc1.weight i.e. Gemm_onnx::Relu_18.weight did not change --- Yay!
fc1.bias i.e. Gemm_onnx::Relu_18.bias did not change --- Yay!
fc2.weight i.e. Gemm_onnx::Relu_20.weight did not change --- Yay!
fc2.bias i.e. Gemm_onnx::Relu_20.bias did not change --- Yay!
fc3.weight i.e. Gemm_22.weight changed --- Yay!
fc3.bias i.e. Gemm_22.bias changed --- Yay!


## VCS: Enter Stage-Right

Now that we've fine tuned the model, let's "commit" this to our VCS.

In [ ]:
from difftools import ModelUpdate, create_diff_file, apply_diff_file
updates = []
for param_name in params_to_train:
    mangled_name = mangle_parameter_name(param_name, model2)
    delta = model2.state_dict()[mangled_name] - imported_pytorch_model.state_dict()[mangled_name]
    updates.append(ModelUpdate(param_name, "dense", delta))

print(len(updates))

diff_file = create_diff_file(updates)
diff_file

4


'17b5abd7678b19252fae2adb69c7a7bf.modeldiff'

And now let's "checkout" the original, non-finetuned model, and apply the diff, allowing us to "train" it without doing any actual work.

In [ ]:
from difftools import apply_diff_file
from onnx_functions import AugumentedOnnxModel

onnx_model = AugumentedOnnxModel("mnist_test_pretrained.onnx")
model_plus_diff = apply_diff_file(onnx_model, diff_file)

And then convert it from Onnx to Pytorch, and do some inference.

In [ ]:
imported_pytorch_model_plus_diff = ConvertModel(model_plus_diff.get_model(), experimental=True)

/Users/montyevans/opt/miniconda3/envs/raffellab/lib/python3.9/site-packages/onnx2pytorch/convert/model.py:147: UserWarning:

Using experimental implementation that allows 'batch_size > 1'.Batchnorm layers could potentially produce false outputs.



In [ ]:
correct_normal, total_normal = evaluate_model(imported_pytorch_model_plus_diff, testloader)
correct_rotated, total_rotated = evaluate_model(imported_pytorch_model_plus_diff, rotated_testloader)

print(f"The imported model classified {correct_normal} digits correctly, out of {total_normal} total, for an accuracy of {correct_normal * 100 / total_normal :.2f}%")
print(f"The imported model classified {correct_rotated} rotated digits correctly, out of {total_rotated} total, for an accuracy of {correct_rotated * 100 / total_rotated :.2f}%")

/Users/montyevans/opt/miniconda3/envs/raffellab/lib/python3.9/site-packages/torch/nn/functional.py:749: UserWarning:

Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.



The imported model classified 9690 digits correctly, out of 10000 total, for an accuracy of 96.90%
The imported model classified 8883 rotated digits correctly, out of 10000 total, for an accuracy of 88.83%


Success! The applied-diff-model achieves the same performance as the fine-tuned-model (because they are, in fact, the same model)!

## More diffs!

In [ ]:
from copy import deepcopy

params_to_train = ["conv2.weight", "conv2.bias"]
masks = get_parameter_masks(params_to_train)

model3 = deepcopy(model2)
optimizer = torch.optim.Adam(model3.parameters())
train_with_mask(model3, optimizer, rotated_trainloader, 1, parameter_masks=masks)

In [141]:
from difftools import ModelUpdate, create_diff_file, apply_diff_file
updates = []
for param_name in params_to_train:
    mangled_name = mangle_parameter_name(param_name, model3)
    delta = model3.state_dict()[mangled_name] - model2.state_dict()[mangled_name]
    updates.append(ModelUpdate(param_name, "dense", delta))

print(len(updates))

diff_file_2 = create_diff_file(updates)
diff_file_2

2


'84f738964db3f87ee4d35778ce6f2edf.modeldiff'

In [142]:
from copy import deepcopy

params_to_train = ["fc2.weight", "fc2.bias"]
masks = get_parameter_masks(params_to_train)

model4 = deepcopy(model3)
optimizer = torch.optim.Adam(model4.parameters())
train_with_mask(model4, optimizer, rotated_trainloader, 1, parameter_masks=masks)

from difftools import ModelUpdate, create_diff_file, apply_diff_file
updates = []
for param_name in params_to_train:
    mangled_name = mangle_parameter_name(param_name, model4)
    delta = model4.state_dict()[mangled_name] - model3.state_dict()[mangled_name]
    updates.append(ModelUpdate(param_name, "dense", delta))

print(len(updates))

diff_file_3 = create_diff_file(updates)
diff_file_3

/Users/montyevans/opt/miniconda3/envs/raffellab/lib/python3.9/site-packages/torch/nn/functional.py:749: UserWarning:

Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.



Loss at epoch 1, batch 375: 0.33
Loss at epoch 1, batch 750: 0.29
Loss at epoch 1, batch 1125: 0.30
Loss at epoch 1, batch 1500: 0.28
Loss at epoch 1, batch 1875: 0.27
Loss at epoch 1, batch 2250: 0.28
Loss at epoch 1, batch 2625: 0.27
Loss at epoch 1, batch 3000: 0.26
Loss at epoch 1, batch 3375: 0.25
Loss at epoch 1, batch 3750: 0.26
2


'e633c8eeac46670ad9836713d3485ca7.modeldiff'

## Unresolved Questions

How can a model's state dict have more keys that the model has parameters? Are the parameters not just the values in the state dict?

There are two items in the imported model's state dict for each parameter in the original model, but besides the names, they seem to be copies of eachother. I am going to go ahead an select one of the copies, and ignore this curiosity. Hopefully this will not be a problem.